In [2]:
# -*- coding: utf-8 -*-

# convolutional autoencoder structure no 3
# a. use of initialisers
# b. use of dilation

# general package dependencies
import numpy as np

from keras.layers import Input # define the input shape for the model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D # for the convnet structure
from keras.models import Model # for the overall definition

from keras.initializers import Constant # bias initialisation
from keras.initializers import TruncatedNormal # kernel initialissation
from keras.layers.advanced_activations import LeakyReLU # activation function (from NSynth)

# load the training dataset
dataset = np.load('spectrogram_test.npz','r')
data_training = dataset['dataset']
data_training = np.transpose(data_training)
print('Training dataset with shape', data_training.shape)
print('Batch size:', data_training.shape[0])

# Convolutional autoencoder structure using the Keras Model API
# define input shape
input_img = Input(shape=(500,128,1))
print('Some information about tensor expected shapes')
print('Input tensor shape:', input_img.shape)

# Convolutional autoencoder definition
# define encoder convnet
x = Conv2D(filters=128,kernel_size=(4,4),activation='relu',padding='same',bias_initializer=Constant(0.1),kernel_initializer=TruncatedNormal())(input_img)
x = MaxPooling2D(pool_size=(2,2),strides=(2),padding='same')(x)
x = Conv2D(filters=256,kernel_size=(2,2),activation='relu',padding='same',bias_initializer=Constant(0.1),kernel_initializer=TruncatedNormal())(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2),padding='same')(x)
encoded = Conv2D(filters=512,kernel_size=(5,5),activation='relu',padding='same',bias_initializer=Constant(0.1),kernel_initializer=TruncatedNormal())(x)
print('Encoded representation tensor shape:', encoded.shape)

# define decoder convnet
x = Conv2D(filters=256,kernel_size=(5,5),activation='relu',padding='same',dilation_rate=1,bias_initializer=Constant(0.1),kernel_initializer=TruncatedNormal())(encoded)
x = UpSampling2D(size=(2,2))(x)
x = Conv2D(filters=128,kernel_size=(2,2),activation='relu',padding='same',dilation_rate=2,bias_initializer=Constant(0.1),kernel_initializer=TruncatedNormal())(x)
x = UpSampling2D(size=(2,2))(x)
decoded = Conv2D(filters=1,kernel_size=(4,4),activation='relu',padding='same',dilation_rate=4,bias_initializer=Constant(0.1),kernel_initializer=TruncatedNormal())(x)
print('Decoded representation tensor shape:', decoded.shape)

# define overal autoencoder model
cae = Model(inputs=input_img, outputs=decoded)
cae.compile(optimizer='adam', loss='mse')

# check for equal size
# obs: the missing value is the batch_size
if input_img.shape[1:] != decoded.shape[1:]: print('alert: in/out dimension mismatch')

Training dataset with shape (1145, 500, 128)
Batch size: 1145
Some information about tensor expected shapes
Input tensor shape: (?, 500, 128, 1)
Encoded representation tensor shape: (?, 125, 32, 512)


AttributeError: 'Tensor' object has no attribute 'output'

In [ ]:
# reshape the training set in 4-dimension tensor
data_training = np.reshape(data_training, (len(data_training), data_training.shape[1], data_training.shape[2], 1))
print('Data training reshaping in tensor of shape', data_training.shape)

# Autoencoder training
cae.fit(data_training,data_training,
        epochs=10,
        batch_size=16, # minibatch of 16 for memory optimisation
        #callbacks=[TensorBoard(log_dir='/tmp/autoencoder')]
       )


# Saving the weights
import h5py
cae.save('cae_dtu_structure3.h5')